In [20]:
import tensorflow as tf

In [21]:
tf.__version__

'2.8.0'

In [22]:
#ses dosyalarının özelliklerini çıkarabilmek için librosa kullanılacak
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import numpy as np

from tqdm import tqdm

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam

In [24]:
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

Tek bir ses öreneği üzerinde deneme

In [25]:
audio_file_path="UrbanSound8K/audio/fold1/101415-3-0-2.wav"
#ses dosyasını dijital hale getiriyoruz(x,sr)
librosa_audio_data,librosa_sample_rate=librosa.load(audio_file_path)

Mel frekans ölçeği, insan kulağının ses frekanslarındaki değişimi algılayışını gösteren bir ölçektir. MFCC(Mel-Frequency Cepstral Coefficients), ses sinyalinin kısa zamanlı güç spektrumunun Mel ölçeği üzerindeki ifadesidir.

Frekanstan Mel ölçeğine dönüşüm formülü;

M ═ 1125 × ln(1+(f÷700))

In [26]:
mfccs=librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate,n_mfcc=40)

In [27]:
mfccs.shape

(40, 173)

Şimdi toplu halde verilerin özniteliklerini çıkartacağım

In [28]:
audio_dataset_path="UrbanSound8K/audio/"
metadata=pd.read_csv("UrbanSound8K/metadata/UrbanSound8K.csv")
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [32]:
def feature_extractor(file):
    audio,sample_rate=librosa.load(file_name,res_type="kaiser_fast")
    mfccs_features=librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [33]:
import os
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path),"fold"+str(row["fold"])+"/",str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=feature_extractor(file_name)
    extracted_features.append([data,final_class_labels])

3554it [03:25, 18.60it/s]C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1323
  return f(*args, **kwargs)
8326it [08:19, 24.52it/s]C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1103
  return f(*args, **kwargs)
C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1523
  return f(*args, **kwargs)
8732it [08:43, 16.69it/s]


In [34]:
extracted_features_df=pd.DataFrame(extracted_features,columns=["features","class"])
extracted_features_df.head()

,features,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.52066, 52.00812, -...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036354, ...",children_playing
4,"[-446.60352, 113.68541, -52.402214, 60.302044,...",children_playing


In [35]:
x=np.array(extracted_features_df["features"].tolist())
y=np.array(extracted_features_df["class"].tolist())

In [36]:
x.shape

(8732, 40)

In [37]:
y.shape

(8732,)

In [38]:
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [39]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [41]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=0)

In [42]:
xtrain

array([[-1.3110471e+02,  1.1250591e+02, -2.2574696e+01, ...,
         3.2466526e+00, -1.3690238e+00,  2.7557547e+00],
       [-1.3670342e+01,  9.1085083e+01, -7.7927332e+00, ...,
        -3.2530503e+00, -5.2774525e+00, -1.5569715e+00],
       [-4.9871544e+01,  2.6535282e-01, -2.0500937e+01, ...,
         2.8545945e+00, -1.6092046e+00,  3.5248058e+00],
       ...,
       [-4.2701236e+02,  9.2623047e+01,  3.1293976e+00, ...,
         7.4264127e-01,  7.3349077e-01,  7.1100914e-01],
       [-1.4575461e+02,  1.3626578e+02, -3.3515522e+01, ...,
         1.4681194e+00, -2.0091701e+00, -8.8218188e-01],
       [-4.2103134e+02,  2.1065454e+02,  3.4906607e+00, ...,
        -5.3888674e+00, -3.3713605e+00, -1.5665114e+00]], dtype=float32)

In [43]:
num_labels=10

In [44]:
model=Sequential()
model.add(Dense(125,input_shape=(40,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#2.gizli katman
model.add(Dense(250))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#3.gizli katman
model.add(Dense(125))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#output kısmı
model.add(Dense(num_labels))
model.add(Activation("softmax"))

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               5125      
                                                                 
 activation (Activation)     (None, 125)               0         
                                                                 
 dropout (Dropout)           (None, 125)               0         
                                                                 
 dense_1 (Dense)             (None, 250)               31500     
                                                                 
 activation_1 (Activation)   (None, 250)               0         
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                                 
 dense_2 (Dense)             (None, 125)               3

In [51]:
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

In [52]:
epochscount=300
num_batch_size=32

In [53]:
model.fit(xtrain,ytrain,batch_size=num_batch_size,epochs=epochscount,validation_data=(xtest,ytest),verbose=1)

Epoch 1/300
219/219 [==============================] - 1s 3ms/step - loss: 9.5413 - accuracy: 0.1246 - val_loss: 2.2954 - val_accuracy: 0.1053
Epoch 2/300
219/219 [==============================] - 0s 2ms/step - loss: 2.4478 - accuracy: 0.1256 - val_loss: 2.2824 - val_accuracy: 0.1053
Epoch 3/300
219/219 [==============================] - 0s 2ms/step - loss: 2.3082 - accuracy: 0.1435 - val_loss: 2.2565 - val_accuracy: 0.1334
Epoch 4/300
219/219 [==============================] - 0s 2ms/step - loss: 2.2540 - accuracy: 0.1479 - val_loss: 2.1916 - val_accuracy: 0.1723
Epoch 5/300
219/219 [==============================] - 0s 2ms/step - loss: 2.2134 - accuracy: 0.1613 - val_loss: 2.1210 - val_accuracy: 0.1855
Epoch 6/300
219/219 [==============================] - 1s 2ms/step - loss: 2.1839 - accuracy: 0.1717 - val_loss: 2.0925 - val_accuracy: 0.1866
Epoch 7/300
219/219 [==============================] - 1s 2ms/step - loss: 2.1615 - accuracy: 0.1788 - val_loss: 2.0797 - val_accuracy: 0.1963

In [54]:
validation_test_set_accuracy=model.evaluate(xtest,ytest,verbose=0)
print(validation_test_set_accuracy[1])

0.8248425722122192


In [58]:
xtest[1]

array([-466.17957   ,    1.0950246 ,  -34.01389   ,   35.339348  ,
        -14.88148   ,  -19.12843   ,   -0.581684  ,  -16.130579  ,
        -21.339075  ,    7.6736345 ,  -29.16449   ,  -18.950253  ,
         -2.9579992 ,   -8.16233   ,  -15.153101  ,   -6.6048055 ,
         -7.5685973 ,    9.340646  ,   14.4331    ,   21.934181  ,
         20.861397  ,    1.3340123 ,  -19.228804  ,   -4.630231  ,
         -1.0564746 ,    3.215267  ,   -6.984281  ,  -16.414577  ,
        -10.0286455 ,   13.009954  ,    0.5334608 ,  -23.843391  ,
        -15.267321  ,    9.245734  ,   10.367626  ,   -0.58320105,
         -1.2624053 ,   17.700016  ,   13.847462  ,   -5.1862826 ],
      dtype=float32)

In [61]:
filename="UrbanSound8K/audio/1.wav"
sound_signal,sample_rate=librosa.load(filename,res_type="kaiser_fast")
mfcc_features=librosa.feature.mfcc(y=sound_signal,sr=sample_rate,n_mfcc=40)
mfccs_scaled_features=np.mean(mfcc_features.T,axis=0)

In [62]:
mfccs_scaled_features

array([-211.69771   ,  136.81721   ,  -51.624107  ,   -5.330061  ,
        -25.640839  ,   15.286624  ,    5.487302  ,   23.88134   ,
        -13.51877   ,   -3.1207569 ,   -9.847641  ,    5.5097537 ,
        -17.601685  ,   -3.7121212 ,   -7.3693094 ,    2.9336474 ,
         -8.740864  ,    0.36794883,  -12.837932  ,   -1.8216788 ,
         -2.5387962 ,    2.5651174 ,   -2.5372047 ,   -0.22441551,
         -1.336262  ,    4.510054  ,   -1.1887121 ,   -3.3464003 ,
         -4.1703806 ,   -1.7491632 ,   -0.80172765,    0.59084535,
         -1.1156591 ,   -0.5076816 ,    0.30087218,   -1.8190004 ,
         -0.4655987 ,   -2.5703588 ,   -3.1206923 ,   -0.49080592],
      dtype=float32)

In [63]:
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)

In [64]:
mfccs_scaled_features.shape

(1, 40)

In [66]:
result_array=model.predict(mfccs_scaled_features)

In [67]:
result_array

array([[0.00223602, 0.00729365, 0.04023099, 0.1635184 , 0.01344741,
        0.00981442, 0.01863409, 0.00121669, 0.7153332 , 0.02827516]],
      dtype=float32)

In [68]:
result_classes=["klima","korna","çocuk sesleri","köpek havlaması",
               "sondaj","motor sesi","silah sesi","darbeli kırıcı","siren","sokak müziği"]

In [71]:
result=np.argmax(result_array[0])
result_classes[result]

'siren'

In [72]:
from IPython.display import Audio
Audio("UrbanSound8K/audio/1.wav")